In [1]:
file=open("puzzle.txt","r")

In [2]:
puzzle=file.read()

In [3]:
print(puzzle)

705600103001070900032001740000890061200000004510034000083700590009050600607002408


In [4]:
import numpy as np

In [76]:
def printPuzzle(stringPuzzle):
    puzzle=np.zeros((9,9))
    for i in range(9):
        for j in range(9):
            puzzle[i][j]=stringPuzzle[i*9+j]

    puzzle=puzzle.astype(int)
    print(puzzle)

In [77]:
printPuzzle(puzzle)

[[7 0 5 6 0 0 1 0 3]
 [0 0 1 0 7 0 9 0 0]
 [0 3 2 0 0 1 7 4 0]
 [0 0 0 8 9 0 0 6 1]
 [2 0 0 0 0 0 0 0 4]
 [5 1 0 0 3 4 0 0 0]
 [0 8 3 7 0 0 5 9 0]
 [0 0 9 0 5 0 6 0 0]
 [6 0 7 0 0 2 4 0 8]]


In [6]:
#Initial set up for specific puzzle
def setup(puzzle):
    exactcover=np.zeros((729,324))
    exactcover=exactcover.astype(int)

    for i in range (81):
        for j in range (9):
            exactcover[j+i*9][i]=1

    for n in range (9): 
        for m in range (9):
            for k in range (9):  
                exactcover[k+m*9+81*n][81+k+9*n]=1
                exactcover[k+m*9+81*n][162+k+9*m]=1

    for p in range (3):
        for q in range (3):
            for n in range (3):
                for m in range (3):
                    for k in range (9):  
                        exactcover[k+m*9+27*n+81*q+243*p][243+k+9*n+27*p]=1

    for k in range (9):
        for j in range (9):
            m=int(puzzle[k][j])
            if m>0:
                row=k*81+j*9+m-1
                for i in range (324):
                    if exactcover[row][i]==1:
                        for j in range (729):
                            if exactcover[j][i]==1 and j!=row:
                                exactcover[j]=0
    return exactcover

In [7]:
def showMatrix(matrix):
    f=open("exactcover.txt","w+")
    for i in range(729):
        for j in range(324):
            test=np.array2string(matrix[i][j])
            f.write(test)
        f.write("\n")

In [9]:
#exactcover=setup(original)

In [11]:
#showMatrix(exactcover)

In [8]:
def selectRow(problem):
    minOnesCount=list()
    for i in range (np.shape(problem)[1]):
        currentOnes=np.sum(problem[:,i])
        minOnesCount.append(currentOnes)
    column=minOnesCount.index(min(minOnesCount))
    for j in range (np.shape(problem)[0]):
        if problem[j,column]==1:
            return j
    return "null"

In [9]:
def editProblem(row, newProblem):
    deleteRows=list()
    deleteColumns=list()
    for i in range (np.shape(newProblem)[1]):
        if newProblem[row][i]==1:
            deleteColumns.append(i)
            for j in range (np.shape(newProblem)[0]):
                if newProblem[j][i]==1:
                    deleteRows.append(j)
    for n in deleteRows:
        newProblem[n,:]=0
    for m in reversed(deleteColumns):
        newProblem=np.delete(newProblem, m, 1)
    return newProblem

In [10]:
def resetProblem(Problem, previouslySelected):
    for i in previouslySelected:
        Problem[i,:]=0
    return Problem

In [72]:
def solving(problem, exactcover, state="start", previouslySelected=list(), selectedRows=list(), solution = list()):
    
    if (state=="start" or state=="retry"):
        for i in range (np.shape(exactcover)[1]):
            if np.max(exactcover[:,i])==0:
                state="impossible"
    
    if state=="impossible":
        return "impossible"
    
    row=selectRow(exactcover)
    
    if (state=="start" or state=="retry"): 
        previouslySelected.append(row)
    
    workingExactcover=editProblem(row,exactcover)   
    selectedRows.append(row)
    
    if np.size(workingExactcover)==0:
        state="done"
    else:
        state="continue"
    
    for i in range (np.shape(workingExactcover)[1]):
        if np.max(workingExactcover[:,i])==0:
            state="retry"
    
    if state=="retry":
        selectedRows=list() 
        workingExactcover=resetProblem(setup(problem), previouslySelected)
        retryResult=solving(problem, workingExactcover, state, previouslySelected, selectedRows)
        if retryResult=="impossible":
            return "impossible"
        
    if state=="continue":
        solving(problem, workingExactcover, state, previouslySelected, selectedRows)
    
    
    if state=="done":
        selectedRows.sort()
        for x in selectedRows:
            solution.append(x%9+1)
        
    return solution

In [79]:
initialExactcover=setup(original)
solution=solving(original, initialExactcover)
printPuzzle(solution)

[[7 4 5 6 2 9 1 8 3]
 [8 6 1 4 7 3 9 2 5]
 [9 3 2 5 8 1 7 4 6]
 [3 7 4 8 9 5 2 6 1]
 [2 9 8 1 6 7 3 5 4]
 [5 1 6 2 3 4 8 7 9]
 [1 8 3 7 4 6 5 9 2]
 [4 2 9 3 5 8 6 1 7]
 [6 5 7 9 1 2 4 3 8]]
